## Imports & preparatory steps

In [1]:
import os
import shutil
import os.path as osp
import yaml
import pickle
import logging

logging.basicConfig(level=logging.DEBUG)

import torch
from torch import nn
import torch.nn.functional as F

# !!! DEBUG !!! Nastavení seedu
import random
import numpy as np
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
# Pokud používáte GPU:
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # Pro více GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

from accelerate import Accelerator
# from accelerate.utils import LoggerType

from transformers import AdamW
from transformers import AlbertConfig, AlbertModel
from accelerate import DistributedDataParallelKwargs

from model import MultiTaskModel
from dataloader import build_dataloader
from utils import length_to_mask# , scan_checkpoint

from datasets import load_from_disk
# from torch.utils.tensorboard import SummaryWriter

from torch import __version__ as torch_version
from platform import python_version

# Check CUDA is available
assert torch.cuda.is_available(), "CPU training is not allowed."

# Check the number of CPUs
N_CPUS = int(os.environ["PBS_NUM_PPN"])

# Limit CPU operation in pytorch to `N_CPUS`
torch.set_num_threads(N_CPUS)
# torch.set_num_interop_threads(N_CPUS) !!! DEBUG !!!

# Set username
USER = os.environ["USER"]

# GPU
N_GPUS = torch.cuda.device_count()
# nvidia_smi.nvmlInit()

print(" > Computational resources...")
print(f" | > Number of CPUs: {N_CPUS}")
print(f" | > Number of GPUs: {N_GPUS}")
# for idx in range(n_gpus):
#    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(idx)
#    print(f" | > Device {idx}: {nvidia_smi.nvmlDeviceGetName(handle)}")
print(" > Python & module versions...")
print(f" | > Python:    {python_version()}")
print(f" | > PyTorch:   {torch_version}")

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
INFO:datasets:PyTorch version 2.1.2+cu121 available.
INFO:datasets:TensorFlow version 2.14.0+nv23.12 available.
INFO:datasets:JAX version 0.4.23 available.


 > Computational resources...
 | > Number of CPUs: 1
 | > Number of GPUs: 1
 > Python & module versions...
 | > Python:    3.10.12
 | > PyTorch:   2.1.2+cu121


## Settings

In [2]:
# Check interactive mode
INTERACTIVE_MODE = bool("JupyterLab" in os.environ["PBS_JOBNAME"])

### Papermill settings

In [3]:
log_dir = "models/test"
mixed_precision = "fp16"
data_folder = "datasets/cz-wikipedia.processed"
batch_size = 4
save_interval = 50000
log_interval = 1
num_process = 1 # number of GPUs
# num_steps = 1000000
num_steps = 12

dataset_params = {
    "tokenizer": "fav-kky/FERNET-C5",
    "token_separator": " ", # token used for phoneme separator (space)
    "token_mask": "M", # token used for phoneme mask (M)
    "word_separator": 18065, # token used for word separator (|)
    "token_maps": "token_maps.pkl", # token map path
    "symbol_dict_path": "symbol_dict.csv",  # symbol definition dictionary
    
    "max_mel_length": 512, # max phoneme length
    
    "word_mask_prob": 0.15, # probability to mask the entire word
    "phoneme_mask_prob": 0.1, # probability to mask each phoneme
    "replace_prob": 0.2, # probablity to replace phonemes
}
    
model_params = {
    "vocab_size": 81, # 178
    "hidden_size": 768,
    "num_attention_heads": 12,
    "intermediate_size": 2048,
    "max_position_embeddings": 512,
    "num_hidden_layers": 12,
    "dropout": 0.1,
}

## Copy data to scratch dir

In [4]:
scratch_dir = os.environ["SCRATCHDIR"]
if not INTERACTIVE_MODE:
    # Copy dataset
    # Prepare dataset dir in the scratch
    print(f"> Copying data to local scratch: {scratch_dir}")
    local_data_folder = os.path.join(scratch_dir, os.path.basename(data_folder))
    shutil.copytree(data_folder, local_data_folder)
    # local_tokenizer_folder = os.path.join(scratch_dir, os.path.basename(dataset_params["tokenizer"]))
    # shutil.copytree(dataset_params["tokenizer"], local_tokenizer_folder)
    # Store the scratch dataset so that it is used for training
    data_folder = local_data_folder
    # dataset_params["tokenizer"] = local_tokenizer_folder

## Create/update config file

In [5]:
config = {
    "log_dir": log_dir,
    "mixed_precision": mixed_precision,
    "data_folder": data_folder,
    "batch_size": batch_size,
    "save_interval": save_interval,
    "log_interval": log_interval,
    "num_process": num_process, # number of GPUs
    "num_steps": num_steps,
    "dataset_params": dataset_params,
    "model_params": model_params,
}

config_file = os.path.join(scratch_dir, "config.yml")
# Write to a YAML file
with open(config_file, 'w') as file:
    yaml.dump(config, file)

## Run training script

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config['dataset_params']['tokenizer'])
logging.info('Vocab size: %d', tokenizer.vocab_size)

dataset = load_from_disk(config["data_folder"])
logging.info(dataset)

# Load token maps
with open(config['dataset_params']['token_maps'], 'rb') as handle:
    token_maps = pickle.load(handle)
logging.info('Token maps size: %d', len(token_maps))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /fav-kky/FERNET-C5/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
INFO:root:Vocab size: 99999
INFO:root:Dataset({
    features: ['input_ids', 'phonemes'],
    num_rows: 6248499
})
INFO:root:Token maps size: 95272


In [7]:
criterion = nn.CrossEntropyLoss()

best_loss = float('inf')  # best test loss
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

num_steps = config['num_steps']
log_interval = config['log_interval']
save_interval = config['save_interval']

In [8]:
def train():
    global dataset, config, N_CPUS

    ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
    
    curr_steps = 0
    log_dir = config['log_dir']

    if not osp.exists(log_dir):
        os.makedirs(log_dir, exist_ok=True)
    shutil.copy(config_file, osp.join(log_dir, osp.basename(config_file)))
    
    batch_size = config["batch_size"]
    train_loader = build_dataloader(
        dataset,
        validation=True,        # !!! DEBUG !!!
        batch_size=batch_size,
        num_workers=N_CPUS,
        # num_workers=1,
        dataset_config=config['dataset_params']
    )

    albert_base_configuration = AlbertConfig(**config['model_params'])
    bert = AlbertModel(albert_base_configuration)

    num_vocab = 1 + max([m['token'] for m in token_maps.values()])
    print(f'Num. vocab: {num_vocab}')

    bert = MultiTaskModel(
        bert,
        num_vocab=num_vocab,
        # num_vocab=tokenizer.vocab_size,
        num_tokens=config['model_params']['vocab_size'],
        hidden_size=config['model_params']['hidden_size']
    )
    
    load = True
    try:
        ckpts = []
        for f in os.listdir(log_dir):
            if f.startswith("step_"): ckpts.append(f)

        iters = [int(f.split('_')[-1].split('.')[0]) for f in ckpts if os.path.isfile(os.path.join(log_dir, f))]
        iters = sorted(iters)[-1]
    except:
        iters = 0
        load = False
    
    optimizer = AdamW(bert.parameters(), lr=1e-4)
    accelerator = Accelerator(
        mixed_precision=config['mixed_precision'],
        split_batches=True,
        kwargs_handlers=[ddp_kwargs]
    )
    
    if load:
        checkpoint = torch.load(log_dir + "/step_" + str(iters) + ".t7", map_location='cpu')
        state_dict = checkpoint['net']
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v

        bert.load_state_dict(new_state_dict, strict=False)
        
        accelerator.print('Checkpoint loaded.')
        optimizer.load_state_dict(checkpoint['optimizer'])
    
    bert, optimizer, train_loader = accelerator.prepare(
        bert,
        optimizer,
        train_loader
    )

    accelerator.print('Start training...')

    # Training is stopped after the defined number of steps
    # => just set up a high upper bound in the range
    for epoch in range(1, 1000000):
        running_loss = 0
    
        for batch in train_loader:        
            curr_steps += 1
            
            word_ids, gt_ph_ids, masked_ph_ids, input_lengths, masked_indices = batch
            # word_ids: list of word IDs;  each word = word IDs repeated word_length_in_phonemes times
            # gt_ph_ids: list of ground-truth phoneme label IDs incl. punctuation and spaces between words
            # masked_ph_ids: list of (masked) phoneme IDs incl. punctuation and spaces between words
            # input_lengths: list of lengths of input phoneme strings
            # masked_index: list of masked phoneme indices

            text_mask = length_to_mask(torch.Tensor(input_lengths)).to('cuda')
            tokens_pred, words_pred = bert(masked_ph_ids, attention_mask=(~text_mask).int())
            
            # P2G loss
            loss_vocab = 0
            for _s2s_pred, _text_input, _text_length, _masked_indices in zip(words_pred, word_ids, input_lengths, masked_indices):
                # loss_vocab += criterion(_s2s_pred[:_text_length], _text_input[:_text_length])
                curr_loss = criterion(_s2s_pred[:_text_length], _text_input[:_text_length])
                loss_vocab += curr_loss
                
                # print(f"words_ids:\n{_text_input}")
                # print(f"words_pred:\n{_s2s_pred}")
                # print(f"tokens_pred:\n{tokens_pred}")
                # print(f"loss_vocab: {curr_loss}")
                
            loss_vocab /= word_ids.size(0)
            # print(f"words size: {word_ids.size(0)}")
            # print(f"avg loss_vocab: {loss_vocab}")
            
            # Phoneme loss
            loss_token = 0
            sizes = 1
            for _s2s_pred, _text_input, _text_length, _masked_indices in zip(tokens_pred, gt_ph_ids, input_lengths, masked_indices):
                if len(_masked_indices) > 0:
                    _text_input = _text_input[:_text_length][_masked_indices]
                    loss_tmp = criterion(_s2s_pred[:_text_length][_masked_indices], _text_input[:_text_length]) 
                    loss_token += loss_tmp
                    sizes += 1
            loss_token /= sizes

            # Total loss
            loss = loss_vocab + loss_token

            optimizer.zero_grad()
            accelerator.backward(loss)
            optimizer.step()

            running_loss += loss.item()

            iters += 1
            if (iters+1) % log_interval == 0:
                accelerator.print ('Epoch %2d: Step [%d/%d], Loss: %.5f, Vocab Loss: %.5f, Token Loss: %.5f'
                        %(epoch, iters+1, num_steps, running_loss/log_interval, loss_vocab, loss_token))
                running_loss = 0
                
            if (iters+1) % save_interval == 0:
                accelerator.print('Saving...')

                state = {
                    'net':  bert.state_dict(),
                    'step': iters,
                    'optimizer': optimizer.state_dict(),
                }

                accelerator.save(state, log_dir + '/step_' + str(iters + 1) + '.t7')

            if curr_steps > num_steps:
                return

In [9]:
print(dataset[0])

{'input_ids': [1845, 10543, 2541, 32, 2381, 5402, 71923, 8110, 8, 36066, 3933, 6639, 111, 53, 72793, 11905, 25758, 19, 1899, 64154, 9, 19, 3537, 32, 703, 111, 24, 91222, 34433, 9, 10545, 325, 28646, 10], 'phonemes': ['ve', 'dvacAtEm', 'stoleTI', 'se', 'zaCal', 'nAzef', 'filozofije', 'pouZIvat', 'f', 'SirSIm', 'slova', 'smislu', 'i', 'pro', 'vipracovanE', 'sistEmi', 'nAboZenskE', 'a', 'Skoli', 'moudrosTi', ',', 'a', 'mluvI', 'se', 'tedi', 'i', 'o', 'filozofiji', 'indickE', ',', 'CInskE', 'nebo', 'africkE', '.']}


In [10]:
train()
# from accelerate import notebook_launcher
# while True:
#    notebook_launcher(train, args=(), num_processes=N_GPUS, use_port=33389)

Num. vocab: 95269


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start training...


DEBUG:dataloader:phwords (34): ['ve', 'dvacAtEm', 'stoleTI', 'se', 'zaCal', 'nAzef', 'filozofije', 'pouZIvat', 'f', 'SirSIm', 'slova', 'smislu', 'i', 'pro', 'vipracovanE', 'sistEmi', 'nAboZenskE', 'a', 'Skoli', 'moudrosTi', ',', 'a', 'mluvI', 'se', 'tedi', 'i', 'o', 'filozofiji', 'indickE', ',', 'CInskE', 'nebo', 'africkE', '.']
DEBUG:dataloader:word IDs (34): [1845, 10543, 2541, 32, 2381, 5402, 71923, 8110, 8, 36066, 3933, 6639, 111, 53, 72793, 11905, 25758, 19, 1899, 64154, 9, 19, 3537, 32, 703, 111, 24, 91222, 34433, 9, 10545, 325, 28646, 10]
DEBUG:dataloader:masked phsent (196): ve dvacAtEm stoleTI se zaCal nAzef filozofije pouZIvat f MMMMMM slova smislu M pro nEEsvTpdvsa sistEmi nAboZenskE a MMMMM moudrosTi , a tstae se tedi i o liemtAlmar indickE M CInskE nebo africkE .
DEBUG:dataloader:masked ph IDs (196): [70, 53, 26, 52, 70, 49, 51, 27, 68, 30, 61, 26, 67, 68, 63, 60, 53, 44, 34, 26, 67, 53, 26, 74, 49, 28, 49, 60, 26, 62, 27, 74, 53, 54, 26, 54, 57, 60, 63, 74, 63, 54, 57, 58

Epoch  1: Step [2/12], Loss: 15.15687, Vocab Loss: 11.55150, Token Loss: 3.60537
Epoch  1: Step [3/12], Loss: 13.95109, Vocab Loss: 11.22901, Token Loss: 2.72208
Epoch  1: Step [4/12], Loss: 13.82393, Vocab Loss: 10.77517, Token Loss: 3.04876
Epoch  1: Step [5/12], Loss: 13.28237, Vocab Loss: 10.61260, Token Loss: 2.66976


...

Epoch  1: Step [6/12], Loss: 13.52819, Vocab Loss: 10.55093, Token Loss: 2.97726
Epoch  1: Step [7/12], Loss: 13.06085, Vocab Loss: 10.36559, Token Loss: 2.69526
Epoch  1: Step [8/12], Loss: 13.48721, Vocab Loss: 10.33187, Token Loss: 3.15534


DEBUG:dataloader:phwords (11): ['kri', '##stali', '##zuje', 'f', 'kubi', '##ckE', 'ploSJe', 'centrov', '##anE', 'soustavje', '.']
DEBUG:dataloader:word IDs (11): [7821, 5911, 2028, 8, 34669, 302, 34724, 71402, 1209, 61098, 10]
DEBUG:dataloader:masked phsent (52): kristalizuje f kubickE ploSJe MMMMMMManE MMMMMMMMM .
DEBUG:dataloader:masked ph IDs (52): [59, 66, 57, 67, 68, 49, 60, 57, 74, 69, 58, 53, 26, 54, 26, 59, 69, 50, 57, 51, 59, 30, 26, 64, 60, 63, 43, 35, 53, 26, 37, 37, 37, 37, 37, 37, 37, 49, 62, 30, 26, 37, 37, 37, 37, 37, 37, 37, 37, 37, 26, 1]
DEBUG:dataloader:GT ph IDs (52): [59, 66, 57, 67, 68, 49, 60, 57, 74, 69, 58, 53, 26, 54, 26, 59, 69, 50, 57, 51, 59, 30, 26, 64, 60, 63, 43, 35, 53, 26, 51, 53, 62, 68, 66, 63, 70, 49, 62, 30, 26, 67, 63, 69, 67, 68, 49, 70, 58, 53, 26, 1]
DEBUG:dataloader:word IDs (52): [7821, 7821, 7821, 5911, 5911, 5911, 5911, 5911, 2028, 2028, 2028, 2028, 18065, 8, 18065, 34669, 34669, 34669, 34669, 302, 302, 302, 18065, 34724, 34724, 34724, 3472

Epoch  1: Step [9/12], Loss: 13.13899, Vocab Loss: 10.24442, Token Loss: 2.89457
Epoch  1: Step [10/12], Loss: 13.00294, Vocab Loss: 10.32618, Token Loss: 2.67676
Epoch  1: Step [11/12], Loss: 12.93000, Vocab Loss: 10.09257, Token Loss: 2.83744


DEBUG:dataloader:phwords (16): ['kosmo', '##logije', 'studuje', 'vesmIr', 'jako', 'celek', 'a', 'zvlASTe', 'jeho', 'vzJik', ',', 'souCasnI', 'a', 'budoucI', 'vIvoj', '.']
DEBUG:dataloader:word IDs (16): [88019, 11029, 21566, 13542, 272, 5963, 19, 7718, 358, 2416, 9, 6308, 19, 6245, 3811, 10]
DEBUG:dataloader:masked phsent (87): hcaaologije studuje vesmIr jako celek M zvlASTe jeho vzJik , souCasnI M MMMMMMM vIvoj .
DEBUG:dataloader:masked ph IDs (87): [56, 51, 49, 49, 63, 60, 63, 55, 57, 58, 53, 26, 67, 68, 69, 52, 69, 58, 53, 26, 70, 53, 67, 61, 34, 66, 26, 58, 49, 59, 63, 26, 51, 53, 60, 53, 59, 26, 37, 26, 74, 70, 60, 27, 43, 44, 53, 26, 58, 53, 56, 63, 26, 70, 74, 35, 57, 59, 26, 2, 26, 67, 63, 69, 28, 49, 67, 62, 34, 26, 37, 26, 37, 37, 37, 37, 37, 37, 37, 26, 70, 34, 70, 63, 58, 26, 1]
DEBUG:dataloader:GT ph IDs (87): [59, 63, 67, 61, 63, 60, 63, 55, 57, 58, 53, 26, 67, 68, 69, 52, 69, 58, 53, 26, 70, 53, 67, 61, 34, 66, 26, 58, 49, 59, 63, 26, 51, 53, 60, 53, 59, 26, 49, 26, 74, 

Epoch  1: Step [12/12], Loss: 12.84237, Vocab Loss: 10.04097, Token Loss: 2.80140
Epoch  1: Step [13/12], Loss: 13.49555, Vocab Loss: 10.63540, Token Loss: 2.86015
Epoch  1: Step [14/12], Loss: 13.30929, Vocab Loss: 10.31566, Token Loss: 2.99363


## Cleanup

In [11]:
if not INTERACTIVE_MODE:
    # Delete all files and subdirectories in the directory
    for filename in os.listdir(scratch_dir):
        file_path = os.path.join(scratch_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # remove file or symlink
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # remove directory
        except Exception as e:
            print(f'Failed to delete {file_path}.')
            print(f'Reason: {e}')